# 생육 기간 예측 프로젝트

### 목적 및 배경
* 한 쌍의 이미지를 입력받아 작물의 생육 기간을 예측하는 모델 개발<br/>
 ※ 이후 환경 변수 데이터가 추가 확보되는 시점에는 작물의 효율적인 생육을 위한 최적의 환경을 도출하는 작업으로 연계도 가능할 것으로 전망

### 데이터 정보 및 학습 진행 방식
* DACON의 "생육 기간 예측 경진대회"에서 제공된 데이터로 진행
* 2개 작물(청경채, 적상추)에 대한 생육 기간 경과일자별 이미지 데이터 저장<br/>
\- 총 753개(청경채 353개, 적상추 400개)
* 작물별 이미지 2장씩을 다양하게 조합하여 2장의 이미지간 경과일을 기준으로 학습 및 평가 진행 예정

### 모델 평가 기준
* RMSE(Root Mean Squared Error)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
import os
import random
from PIL import Image
from glob import glob
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 기본 셋팅

In [ ]:
# seed 고정 함수 정의 => seed 고정을 통해 재현성을 확보하기 위함
def seed_everything(seed):
    # 파이토치 및 넘파이, random 등 관련 모듈에 대한 seed 일괄 설정
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

# seed 고정
seed_everything(2048)

is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')
print(device)

lr = 0.00005
epochs = 3  # 초기 설정 10
batch_size = 4  # 초기 설정 64
valid_batch_size = 4  # 초기 설정 50 => 체크 예정

cuda


# 2개 작물별 데이터 정리 및 DataFrame 저장

#### 데이터프레임 생성 관련 함수 정의

In [ ]:
def get_image_path(root_path=None, resize_path=None):
    # 2개 작물별 디렉토리명 list 추출
    if root_path is None:  # None일 경우, 현재 경로 기준의 하위 디렉토리명 추출
        if resize_path: # resize 옵션값 체크
          bc_directories = glob('./BC_Resize/*')
          lt_directories = glob('./LT_Resize/*')  
        else:
          bc_directories = glob('./BC/*')
          lt_directories = glob('./LT/*')
    else:
        if resize_path: # resize 옵션값 체크
          bc_directories = glob(root_path + 'BC_Resize/*')
          lt_directories = glob(root_path + 'LT_Resize/*')  
        else:
          bc_directories = glob(root_path + 'BC/*')
          lt_directories = glob(root_path + 'LT/*')
    
    # 2개 작물별 모든 이미지 파일 경로 list로 저장
    bc_image_path = []
    for bc_directory in bc_directories:
        images = glob(bc_directory + '/*.png')
        bc_image_path.extend(images)
    lt_image_path = []
    for lt_directory in lt_directories:
        images = glob(lt_directory + '/*.png')
        lt_image_path.extend(images)
    
    return bc_image_path, lt_image_path

def get_dataframe(root_path=None, resize_path=None):
    # 2개 작물별 이미지 파일 경로 list로 저장(BC : 청경채, LT : 적상추)
    bc_image_path, lt_image_path = get_image_path(root_path, resize_path)
    
    # 각 파일명에서 기준일자(day) 정보 추출 및 np.array로 저장 => 파일명에서 마지막 2자리 숫자 정보 추출
    bc_day_array = np.array([int(path.split('.')[-2][-2:]) for path in bc_image_path])
    lt_day_array = np.array([int(path.split('.')[-2][-2:]) for path in lt_image_path])
    
    # 데이터프레임 생성
    bc_df = pd.DataFrame({'image_path' : bc_image_path, 'day' : bc_day_array})
    bc_df['species'] = 'bc'
    lt_df = pd.DataFrame({'image_path' : lt_image_path, 'day' : lt_day_array})
    lt_df['species'] = 'lt'
    
    total_df = pd.concat([bc_df, lt_df]).reset_index(drop=True)
    
    return total_df

#### 데이터프레임 생성 => "total_df"

In [ ]:
TRAIN_FOLDER = '/content/drive/MyDrive/ds_study/data1/open/train_dataset/'  # 구글 코랩 기준 경로
TEST_FOLDER = '/content/drive/MyDrive/ds_study/data1/open/test_dataset/'  # 구글 코랩 기준 경로

total_df = get_dataframe(root_path=TRAIN_FOLDER)
len(total_df), total_df.head()  # 총 753개 데이터 저장 결과 확인

(753,                                           image_path  day species
 0  /content/drive/MyDrive/ds_study/data1/open/tra...    2      bc
 1  /content/drive/MyDrive/ds_study/data1/open/tra...    1      bc
 2  /content/drive/MyDrive/ds_study/data1/open/tra...    3      bc
 3  /content/drive/MyDrive/ds_study/data1/open/tra...    6      bc
 4  /content/drive/MyDrive/ds_study/data1/open/tra...   10      bc)

#### 이미지 사이즈 체크 및 Resize 진행

In [ ]:
img = Image.open(total_df['image_path'][0])
print(img.size)
print(img.mode)

(3280, 2464)
RGB


> 이미지 사이즈가 매우 크므로, 코랩 환경 등을 고려하여 모델에 적용할 사이즈로 resize 및 별도 폴더로 저장 후 진행하고자 함

In [ ]:
# dir_list = ['BC', 'LT']

# # train_dataset 내 Resize 폴더 생성
# for dir in dir_list:
#   os.mkdir(TRAIN_FOLDER + dir + '_Resize')
#   for sub_dir in os.listdir(TRAIN_FOLDER + dir): # 서브 폴더 생성
#     os.mkdir(TRAIN_FOLDER + dir + '_Resize/' + sub_dir)
#     for image_path in glob(TRAIN_FOLDER + dir + '/' + sub_dir + '/*'): # 이미지 resize 및 저장
#       image_file_name = image_path.split('/')[-1]
#       img = Image.open(image_path)
#       img = img.resize((224, 224))
#       img.save(TRAIN_FOLDER + dir + '_Resize/' + sub_dir + '/' + image_file_name)

# # test_dataset 내 Resize 폴더 생성
# for dir in dir_list:
#   os.mkdir(TEST_FOLDER + dir + '_Resize')
#   for sub_dir in os.listdir(TEST_FOLDER + dir): # 서브 폴더 생성
#     os.mkdir(TEST_FOLDER + dir + '_Resize/' + sub_dir)
#     for image_path in glob(TEST_FOLDER + dir + '/' + sub_dir + '/*'): # 이미지 resize 및 저장
#       image_file_name = image_path.split('/')[-1]
#       img = Image.open(image_path)
#       img = img.resize((224, 224))
#       img.save(TEST_FOLDER + dir + '_Resize/' + sub_dir + '/' + image_file_name)

#### 변경된 경로("Resize") 기준 데이터프레임 다시 생성 => "total_df_v2"

In [ ]:
total_df_v2 = get_dataframe(root_path=TRAIN_FOLDER, resize_path=True) # resize 경로로 반영
len(total_df_v2), total_df_v2.head()  # 총 753개 데이터 저장 결과 확인

(753,                                           image_path  day species
 0  /content/drive/MyDrive/ds_study/data1/open/tra...    2      bc
 1  /content/drive/MyDrive/ds_study/data1/open/tra...    1      bc
 2  /content/drive/MyDrive/ds_study/data1/open/tra...    3      bc
 3  /content/drive/MyDrive/ds_study/data1/open/tra...    6      bc
 4  /content/drive/MyDrive/ds_study/data1/open/tra...   10      bc)

In [ ]:
# 이미지 사이즈 재확인 => (224, 224)로 반영된 것을 확인함
img = Image.open(total_df_v2['image_path'][0])
print(img.size)
print(img.mode)

(224, 224)
RGB


# 모델 선언
* 우선 baseline 셋팅 완료 후, 다양한 모델을 적용해 평가 및 테스트 진행 예정

In [ ]:
from torchvision.models import mobilenet_v2

class CompareCNN(nn.Module):
    
    def __init__(self):
        super(CompareCNN, self).__init__()
        self.mobile_net = mobilenet_v2(pretrained=True)
        self.fc_layer = nn.Linear(1000, 1)
    
    def forward(self, input):
        x = self.mobile_net(input)
        output = self.fc_layer(x)
        return output

class CompareNet(nn.Module):
    
    def __init__(self):
        super(CompareNet, self).__init__()
        self.before_net = CompareCNN()
        self.after_net = CompareCNN()
    
    def forward(self, before_input, after_input):
        before = self.before_net(before_input)
        after = self.after_net(after_input)
        delta = after - before
        return delta

In [ ]:
model = CompareNet().to(device)

# 학습을 위한 데이터셋 생성

#### 작물별 이미지 조합 및 train, valid 데이터 저장

In [ ]:
# 동일 작물 내 샘플링된 2개 이미지별 time_delta 산출한 데이터프레임 생성 함수
def get_combination_df(length, species, df):
  before_image_path = []
  after_image_path = []
  time_delta = []

  for i in range(length):
    sample = df[df['species'] == species].sample(2)
    before = sample[sample['day'] == min(sample['day'])].reset_index(drop=True)
    after = sample[sample['day'] == max(sample['day'])].reset_index(drop=True)

    before_image_path.append(before.iloc[0]['image_path'])
    after_image_path.append(after.iloc[0]['image_path'])
    delta = after.iloc[0]['day'] - before.iloc[0]['day']
    time_delta.append(delta)
  
  combination_df = pd.DataFrame({
      'before_image_path' : before_image_path,
      'after_image_path' : after_image_path,
      'time_delta' : time_delta
  })

  combination_df['species'] = species

  return combination_df

In [ ]:
data_length = 50  # 추출할 조합의 개수
valid_size = 0.2  # validation 데이터 비율
train_data_length = int(data_length*(1-valid_size))

# 이미지 조합 데이터프레임 생성
bc_comb_df = get_combination_df(data_length, 'bc', total_df_v2) # total_df_v2(resize된 이미지) 기준
lt_comb_df = get_combination_df(data_length, 'lt', total_df_v2) # total_df_v2(resize된 이미지) 기준

# train, valid 각각 분리
bc_train = bc_comb_df[:train_data_length]
lt_train = lt_comb_df[:train_data_length]

bc_valid = bc_comb_df[train_data_length:]
lt_valid = lt_comb_df[train_data_length:]

# train_set, valid_set 저장
train_data = pd.concat([bc_train, lt_train])
valid_data = pd.concat([bc_valid, lt_valid])

In [ ]:
train_data.tail()

,before_image_path,after_image_path,time_delta,species
35,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,11,lt
36,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,3,lt
37,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,39,lt
38,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,11,lt
39,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,7,lt


In [ ]:
valid_data.head()

,before_image_path,after_image_path,time_delta,species
40,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,18,bc
41,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,13,bc
42,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,25,bc
43,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,33,bc
44,/content/drive/MyDrive/ds_study/data1/open/tra...,/content/drive/MyDrive/ds_study/data1/open/tra...,6,bc


#### 데이터셋 만들기

In [ ]:
# 사용자 정의 데이터셋 클래스
class ImageDataset(Dataset):
  def __init__(self, combination_df, is_test=None):
    self.combination_df = combination_df
    self.transform = transforms.Compose([
      # transforms.Resize(224),  # 이미지 resize 기적용 완료
      transforms.ToTensor()
    ])
    self.is_test = is_test
  
  def __len__(self):
    return len(self.combination_df)

  def __getitem__(self, idx):
    before_image = Image.open(self.combination_df.iloc[idx]['before_image_path'])
    after_image = Image.open(self.combination_df.iloc[idx]['after_image_path'])

    before_image = self.transform(before_image)
    after_image = self.transform(after_image)

    if self.is_test:
      return before_image, after_image
    
    time_delta = self.combination_df.iloc[idx]['time_delta']

    return before_image, after_image, time_delta

In [ ]:
train_dataset = ImageDataset(train_data)
valid_dataset = ImageDataset(valid_data)

# 학습 진행

#### 미니 배치 구성

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=valid_batch_size)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in tqdm(range(epochs)):
  for step, (before_image, after_image, time_delta) in tqdm(enumerate(train_loader)):
    before_image = before_image.to(device)
    after_image = after_image.to(device)
    time_delta = time_delta.to(device)

    optimizer.zero_grad()
    logit = model(before_image, after_image)
    train_loss = torch.sum(torch.abs(logit.squeeze(1).float() - time_delta.float())) / torch.LongTensor([batch_size]).squeeze(0).to(device)
    train_loss.backward()
    optimizer.step()

    if step % 1 == 0:
      print(f'============= epoch : {epoch} ================')
      print(f'============= step : {step} =================')
      print('MAE_loss :', train_loss.detach().cpu().numpy())
  
  valid_losses = []
  with torch.no_grad():
    for valid_before, valid_after, time_delta in tqdm(valid_loader):
      valid_before = valid_before.to(device)
      valid_after = valid_after.to(device)
      valid_time_delta = time_delta.to(device)


      logit = model(valid_before, valid_after)
      valid_loss = torch.sum(torch.abs(logit.squeeze(1).float() - valid_time_delta.float())) / torch.LongTensor([valid_batch_size]).squeeze(0).to(device)
      valid_losses.append(valid_loss.detach().cpu())


  print(f'VALIDATION_LOSS MAE : {sum(valid_losses)/len(valid_losses)}')

  0%|          | 0/3 [00:00<?, ?it/s]

0it [00:00, ?it/s]

=============epoch : 0=================
=============step : 0=================
MAE_loss : 19.23135
=============epoch : 0=================
=============step : 1=================
MAE_loss : 12.889063
=============epoch : 0=================
=============step : 2=================
MAE_loss : 14.659756
=============epoch : 0=================
=============step : 3=================
MAE_loss : 16.724632
=============epoch : 0=================
=============step : 4=================
MAE_loss : 14.909941
=============epoch : 0=================
=============step : 5=================
MAE_loss : 12.291484
=============epoch : 0=================
=============step : 6=================
MAE_loss : 5.363242
=============epoch : 0=================
=============step : 7=================
MAE_loss : 5.149747
=============epoch : 0=================
=============step : 8=================
MAE_loss : 10.38232
=============epoch : 0=================
=============step : 9=================
MAE_loss : 3.6235168
====

  0%|          | 0/5 [00:00<?, ?it/s]

VALIDATION_LOSS MAE : 7.86920166015625


...

VALIDATION_LOSS MAE : 6.480809211730957


0it [00:00, ?it/s]

=============epoch : 2=================
=============step : 0=================
MAE_loss : 3.1690311
=============epoch : 2=================
=============step : 1=================
MAE_loss : 4.9575653
=============epoch : 2=================
=============step : 2=================
MAE_loss : 2.726049
=============epoch : 2=================
=============step : 3=================
MAE_loss : 2.1679666
=============epoch : 2=================
=============step : 4=================
MAE_loss : 8.867912
=============epoch : 2=================
=============step : 5=================
MAE_loss : 1.9910932
=============epoch : 2=================
=============step : 6=================
MAE_loss : 3.4028363
=============epoch : 2=================
=============step : 7=================
MAE_loss : 3.5267062
=============epoch : 2=================
=============step : 8=================
MAE_loss : 5.5701146
=============epoch : 2=================
=============step : 9=================
MAE_loss : 6.45322
====

  0%|          | 0/5 [00:00<?, ?it/s]

VALIDATION_LOSS MAE : 6.433302402496338
